In [136]:
'''
Algorithmic Trading Framework
Data is from testnet, and therefore not accurate.

Install dependencies with:

pip install -r requirements.txt
For some functionality you need a valid API-key.  
Export as environment variables.
Afterwards run notebook from same shell.

Bash/Sh/Zsh:

export BINANCE_API_KEY='your_api_key'
export BINANCE_API_SECRET='your_secret_key'

Check
echo $BINANCE_API_KEY
echo $BINANCE_API_SECRET

Powershell:

$env:BINANCE_API_KEY = 'your_api_key'
$env:BINANCE_API_SECRET = 'your_secret_key'

Check
$env:BINANCE_API_KEY
$env:BINANCE_API_KEY
'''
import os
import pandas as pd
import numpy as np
import scipy.optimize as sco
from binance.spot import Spot as Client
pd.set_option('display.max_rows', 80)
# pd.reset_option('display.float_format')

# If your time is off you need to synchronize with timeserver
# Ommit base_url argument to default to api.binance.com

client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_API_SECRET'), base_url='https://testnet.binance.vision')
pd.to_datetime(client.time()['serverTime'], unit='ms')

Timestamp('2022-05-10 22:06:56.037000')

In [137]:
balance = pd.json_normalize(client.account()['balances'])
balance

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [138]:
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT' ]
columns =  ['Open time','Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore' ]
# assets= [pd.DataFrame((c[4] for c in client.klines(symbol, "1d")),columns=[symbol]) for symbol in symbols]
assets = pd.concat(([pd.DataFrame(client.klines(symbol, "1d"), columns=columns) for symbol in symbols]), axis = 1, keys=symbols)
assets

BTCUSDT                                                   \
       Open time            Open             High             Low   
0  1651622400000  38127.16000000   67952.01000000   9000.00000000   
1  1651708800000  39690.01000000   48256.09000000  12000.00000000   
2  1651795200000  36552.97000000   90000.00000000   9500.00000000   
3  1651881600000  35851.70000000  179685.00000000   2000.00000000   
4  1651968000000  35472.40000000  172120.00000000   6988.00000000   
5  1652054400000  34036.87000000   44600.00000000   6392.00000000   
6  1652140800000  30089.10000000   34490.00000000  10000.00000000   

                                                                     \
            Close         Volume     Close time  Quote asset volume   
0  39695.80000000  3289.47888900  1651708799999  128626868.06394152   
1  36551.04000000  3747.85911700  1651795199999  142179013.42520536   
2  35844.54000000  5133.87878000  1651881599999  185618082.67083564   
3  35472.39000000  2648.74837000  1651967999999   94693433.70235498   
4  34035.00000000  5928.73934500  1652054399999  204738551.12075512   
5  30092.43000000  7687.56723900  1652140799999  249999769.04934389   
6  30504.90000000  7901.18807400  1652227199999  247859900.08094468   

                                                ...     XRPUSDT              \
  Number of trades Taker buy base asset volume  ...        High         Low   
0           109403               2070.21409400  ...  0.64990000  0.60830000   
1           136426               2305.80703300  ...  0.65710000  0.58040000   
2           195320               2791.12783400  ...  0.61630000  0.58130000   
3           110202               1439.33242400  ...  0.60340000  0.56940000   
4           216686               3380.01233600  ...  0.58280000  0.55810000   
5           259088               4741.36570900  ...  0.57870000  0.47290000   
6           255970               5046.31565500  ...  0.49490000  0.43330000   

                                                                    \
        Close             Volume     Close time Quote asset volume   
0  0.64610000   7188003.90000000  1651708799999   4497422.00637000   
1  0.59830000  11682556.70000000  1651795199999   7147202.83414000   
2  0.58130000  12824794.10000000  1651881599999   7706389.17155000   
3  0.58180000   4378006.60000000  1651967999999   2579982.36840000   
4  0.56620000   9985261.30000000  1652054399999   5708676.23890000   
5  0.48650000  27793876.20000000  1652140799999  14549295.78418000   
6  0.48640000    447113.80000000  1652227199999    218200.82982000   

                                                                             \
  Number of trades Taker buy base asset volume Taker buy quote asset volume   
0             8638            7020708.90000000             4393175.81487000   
1            13976           11566933.90000000             7074742.84037000   
2            15203           12610238.80000000             7578080.05335000   
3             5475            4097615.10000000             2413970.94234000   
4            10831            9690239.70000000             5539919.86459000   
5            28997           27405138.00000000            14344952.93562000   
6              900             367613.00000000              179875.20292000   

          
  Ignore  
0      0  
1      0  
2      0  
3      0  
4      0  
5      0  
6      0  

[7 rows x 72 columns]

In [174]:
assets = assets.swaplevel(axis=1) # Swapping levels for easier selection
assets = assets.set_index(pd.to_datetime(assets['Close time', 'BTCUSDT'], unit='ms').dt.strftime('%Y-%m-%d')) # Set close time as index, needs improvement
assets.index.name = 'Close time'
assets_close = assets["Close"].copy().astype(float) # Daily close prices
assets_close

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
Close time,,,,,,
2022-05-04,39695.8000,2940.6700,402.6000,106.4000,0.0863,0.6461
2022-05-05,36551.0400,2747.9300,378.5000,96.9000,0.0745,0.5983
2022-05-06,35844.5400,2655.3300,157.0700,95.6000,0.0817,0.5813
2022-05-07,35472.3900,2637.3300,365.5000,94.5000,0.0835,0.5818
2022-05-08,34035.0000,2519.5100,356.0000,94.0000,0.0880,0.5662
2022-05-09,30092.4300,2227.2500,296.4000,76.0000,0.0727,0.4865
2022-05-10,30504.9000,2786.6700,312.7000,76.9000,0.0730,0.4864


In [175]:
returns = assets_close.pct_change().dropna() # Daily Returns
returns.mean(axis = 0)

BTCUSDT   -0.0419
ETHUSDT   -0.0026
BNBUSDT    0.0906
LTCUSDT   -0.0499
TRXUSDT   -0.0224
XRPUSDT   -0.0449
dtype: float64

In [176]:
# Correlation Coefficient
returns.corr()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
BTCUSDT,1.0000,0.7723,0.2991,0.9390,0.8147,0.9622
ETHUSDT,0.7723,1.0000,0.1204,0.6161,0.3404,0.6421
BNBUSDT,0.2991,0.1204,1.0000,0.2304,0.0624,0.4039
LTCUSDT,0.9390,0.6161,0.2304,1.0000,0.8795,0.9788
TRXUSDT,0.8147,0.3404,0.0624,0.8795,1.0000,0.8304
XRPUSDT,0.9622,0.6421,0.4039,0.9788,0.8304,1.0000


In [177]:
riskfree_return = 0.031 # 5 Year Treasury Rate, but testnet resets every month
# Optimal Sharpe Ratio Portfolio (Tangency Portfolio)
asset_qty = len(assets_close.columns)

def portfolio_return(weights):
    return returns.dot(weights.T).mean() * 365.25

def portfolio_risk(weights):
    return returns.dot(weights.T).std() * 365.25

def minimized_sharpe(weights):
    return (riskfree_return - portfolio_return(weights))/portfolio_risk(weights)

equal_weights = np.full(asset_qty, 1/asset_qty)
constraint = ({'type': 'eq', 'fun': lambda x: np.sum(x) -1})
bounds = tuple((0,1) for x in range(asset_qty))

# Sequential Least Squares Programming
optimum = sco.minimize(minimized_sharpe, equal_weights, method='SLSQP', bounds=bounds, constraints=constraint)
np.set_printoptions(suppress = True)
optimum # No surprise with that 157.07$ BNB outlier

     fun: -0.14005273730765355
     jac: array([ 0.06797003,  0.00738359, -0.0001313 ,  0.0810109 ,  0.03614277,
        0.07418953])
 message: 'Optimization terminated successfully'
    nfev: 14
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0., 0., 1., 0., 0., 0.])

In [178]:
# Optimal Weights
pd.options.display.float_format = '{:.4f}'.format
optimal_weights = optimum['x']
optimal_weights = pd.Series(index = assets_close.columns, data = optimal_weights)
optimal_weights

BTCUSDT   0.0000
ETHUSDT   0.0000
BNBUSDT   1.0000
LTCUSDT   0.0000
TRXUSDT   0.0000
XRPUSDT   0.0000
dtype: float64

In [179]:
portfolio_return(optimal_weights)

33.09731003116562

In [180]:
portfolio_risk(optimal_weights)

236.09899147154064

In [181]:
-minimized_sharpe(optimal_weights)

0.14005273730765355

In [182]:
# Tangency Portfolio
returns['TP'] = returns.dot(optimal_weights)
returns

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT,TP
Close time,,,,,,,
2022-05-05,-0.0792,-0.0655,-0.0599,-0.0893,-0.1363,-0.0740,-0.0599
2022-05-06,-0.0193,-0.0337,-0.5850,-0.0134,0.0957,-0.0284,-0.5850
2022-05-07,-0.0104,-0.0068,1.3270,-0.0115,0.0223,0.0009,1.3270
2022-05-08,-0.0405,-0.0447,-0.0260,-0.0053,0.0534,-0.0268,-0.0260
2022-05-09,-0.1158,-0.1160,-0.1674,-0.1915,-0.1731,-0.1408,-0.1674
2022-05-10,0.0137,0.2512,0.0550,0.0118,0.0034,-0.0002,0.0550


In [ ]:
COV = returns.cov()*365.25
COV

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT,TP
BTCUSDT,0.8369,1.7498,3.3801,1.2760,1.5343,0.9130,3.3801
ETHUSDT,1.7498,6.1344,3.6848,2.2669,1.7357,1.6494,3.6848
BNBUSDT,3.3801,3.6848,152.6153,4.2281,1.5875,5.1755,152.6153
LTCUSDT,1.2760,2.2669,4.2281,2.2065,2.6894,1.5080,4.2281
TRXUSDT,1.5343,1.7357,1.5875,2.6894,4.2378,1.7730,1.5875
XRPUSDT,0.9130,1.6494,5.1755,1.5080,1.7730,1.0757,5.1755
TP,3.3801,3.6848,152.6153,4.2281,1.5875,5.1755,152.6153


In [ ]:
COV.iloc[:-1, -1].dot(optimal_weights)

152.61528753970876

In [ ]:
COV.iloc[-1, -1]

152.61528753970876

In [195]:
# Annualised Risk σ, Return, Sharpe & Variance
def annualised_risk_return(returns):
    stats = returns.agg(['mean', 'std']).T
    stats.columns = ['Return', 'Risk']
    stats.Return = stats.Return*365.25 # Crypto exchanges trade every day, including leap days.
    stats.loc[stats.Return < -1, 'Return'] = -1 # Set losses > 100% to -100%
    stats.Risk = stats.Risk * np.sqrt(365.25)
    return stats 
stats = annualised_risk_return(returns) # Risk may not be accurate for mean daily losses > 0.274% ()
stats['Sharpe'] = stats['Return'].sub(riskfree_return)/stats['Risk']
stats['Variance'] = np.power(stats.Risk, 2)
# Systematic & Unsystematic Risk in Variance
stats['Sys. Var.'] = COV.iloc[:,-1]
stats['Unsys. Var.'] = stats['Variance'].sub(stats['Sys. Var.'])
# Normalize == beta
stats['beta'] = stats['Sys. Var.'] / stats.loc['TP', 'Sys. Var.']
# Expected Return
stats["CAPM"] = riskfree_return + (stats.loc["TP", "Return"] - riskfree_return) * stats.beta
stats

,Return,Risk,Sharpe,Variance,Sys. Var.,Unsys. Var.,beta,CAPM
BTCUSDT,-1.0000,0.9148,-1.1270,0.8369,3.3801,-2.5432,0.0221,0.7633
ETHUSDT,-0.9449,2.4768,-0.3940,6.1344,3.6848,2.4496,0.0241,0.8294
BNBUSDT,33.0973,12.3538,2.6766,152.6153,152.6153,0.0000,1.0000,33.0973
LTCUSDT,-1.0000,1.4854,-0.6941,2.2065,4.2281,-2.0215,0.0277,0.9471
TRXUSDT,-1.0000,2.0586,-0.5008,4.2378,1.5875,2.6502,0.0104,0.3750
XRPUSDT,-1.0000,1.0372,-0.9940,1.0757,5.1755,-4.0998,0.0339,1.1523
TP,33.0973,12.3538,2.6766,152.6153,152.6153,-0.0000,1.0000,33.0973
